In [21]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
import sqlite3
import pandas as pd

In [4]:
def tpex_header():
    """網頁瀏覽時, 所帶的 request header 參數, 模仿瀏覽器發送 request"""
    return {
        # "Accept": "application/json, text/javascript, */*; q=0.01",
        # "Accept-Encoding": "gzip, deflate",
        # "Accept-Language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7",
        # "Connection": "keep-alive",
        # "Host": "www.tpex.org.tw",
        "Referer": "https://www.twse.com.tw/zh/trading/historical/mi-index.html",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36",
        # "X-Requested-With": "XMLHttpRequest",
    }
payload = {
    "date":"20250318",
    "type": "ALLBUT0999",
    "response": "json"
}

In [ ]:
re = requests.get("https://www.twse.com.tw/rwd/zh/afterTrading/MI_INDEX", headers=tpex_header(),params=payload)
re.encoding = "utf-8"

In [44]:
def get_general_daily_stock_tw_info(date:str):
    """爬取指定日期的台灣股市資訊"""
    dat = date.split("-")
    
    payload = {
        "date": dat[0]+dat[1]+dat[2],
        "type": "ALLBUT0999",
        "response": "json"
    }
    re = requests.get("https://www.twse.com.tw/rwd/zh/afterTrading/MI_INDEX", headers=tpex_header(),params=payload)
    re.encoding = "utf-8"
    df = re.json()
    try:
        table_df = pd.DataFrame(df["tables"][8]["data"], columns=df["tables"][8]["fields"])
        table_df["漲跌(+/-)"] = table_df["漲跌(+/-)"].astype(str).str.extract(r'>(.*?)<')
        table_df["交易日期"] = date
        return table_df
    except:
        print("No data")
    

In [45]:
data = get_general_daily_stock_tw_info("2025-03-17")

In [46]:
data

,證券代號,證券名稱,成交股數,成交筆數,成交金額,開盤價,最高價,最低價,收盤價,漲跌(+/-),漲跌價差,最後揭示買價,最後揭示買量,最後揭示賣價,最後揭示賣量,本益比,交易日期
0,0050,元大台灣50,"12,068,530","26,331","2,202,388,924",182.55,183.50,181.60,182.05,+,1.55,182.05,23,182.10,11,0.00,2025-03-17
1,0051,元大中型100,"47,526",365,"3,659,967",77.20,77.25,76.65,76.80,+,0.50,76.60,1,76.85,1,0.00,2025-03-17
2,0052,富邦科技,"359,066","1,068","64,809,060",180.40,181.50,179.50,179.55,+,1.45,179.50,3,179.55,1,0.00,2025-03-17
3,0053,元大電子,"11,466",129,"1,113,234",97.40,97.40,96.95,97.10,+,0.75,96.35,2,96.75,1,0.00,2025-03-17
4,0055,元大MSCI金融,"91,514",375,"2,616,775",28.49,28.70,28.27,28.70,+,0.21,28.59,1,28.70,4,0.00,2025-03-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1281,9944,新麗,"89,312",145,"1,789,913",20.10,20.10,19.90,20.05,-,0.05,19.90,5,20.05,4,401.00,2025-03-17
1282,9945,潤泰新,"13,173,779","7,175","496,615,380",37.90,38.10,37.40,37.60,-,0.40,37.60,258,37.65,21,6.44,2025-03-17
1283,9946,三發地產,"345,149",152,"8,592,372",24.95,24.95,24.85,24.90,,0.00,24.90,7,24.95,32,21.65,2025-03-17
1284,9955,佳龍,"492,367",447,"15,078,555",30.80,30.85,30.40,30.40,-,0.25,30.40,5,30.65,7,0.00,2025-03-17


In [47]:
conn = sqlite3.connect("twse_data.db")
cursor = conn.cursor()

In [48]:
cursor.execute("""
CREATE TABLE IF NOT EXISTS stock_data (
    date DATETIME,                 -- 交易日期
    stock_id TEXT,             -- 證券代號
    stock_name TEXT,           -- 證券名稱
    trade_volume INTEGER,      -- 成交股數
    trade_count INTEGER,       -- 成交筆數
    trade_value REAL,          -- 成交金額
    open_price REAL,           -- 開盤價
    high_price REAL,           -- 最高價
    low_price REAL,            -- 最低價
    closing_price REAL,        -- 收盤價
    change_sign TEXT,          -- 漲跌(+/-)
    change_points REAL,        -- 漲跌價差
    final_bid_price REAL,      -- 最後揭示買價
    final_bid_volume INTEGER,  -- 最後揭示買量
    final_ask_price REAL,      -- 最後揭示賣價
    final_ask_volume INTEGER,  -- 最後揭示賣量
    pe_ratio REAL,             -- 本益比
    PRIMARY KEY (date, stock_id)
)
""")

In [49]:
data.to_sql("stock_data", conn, if_exists="replace", index=False)

1286

In [50]:
from datetime import datetime, timedelta

In [52]:
def generate_time(start_date: str, end_date: str):
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    delta = timedelta(days=1)
    date_list = [(start + timedelta(days=i)).strftime("%Y-%m-%d") for i in range((end - start).days + 1)]
    return date_list
    
    

In [53]:
generate_time("2025-03-01", "2025-03-17")

['2025-03-01',
 '2025-03-02',
 '2025-03-03',
 '2025-03-04',
 '2025-03-05',
 '2025-03-06',
 '2025-03-07',
 '2025-03-08',
 '2025-03-09',
 '2025-03-10',
 '2025-03-11',
 '2025-03-12',
 '2025-03-13',
 '2025-03-14',
 '2025-03-15',
 '2025-03-16',
 '2025-03-17']